In [1]:
# Params
input_size = (300,300)
batch_size = 100
num_workers = 5
num_classes = 2
learning_rate = 1e-03
SGD_momentum = 0.9
epochs = 20

# Training an image classifier

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


## 1. Loading and normalizing MH dataset

In [3]:
from dataset_MH import *

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [4]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb1 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='train', input_size=input_size, augmentation=True)
mhdb2 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='train', input_size=input_size, augmentation=True)
mhdb3 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='train', input_size=input_size, augmentation=True)
mhdb4 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='train', input_size=input_size, augmentation=True)

mhdb5 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='val', input_size=input_size, augmentation=True)
mhdb6 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='val', input_size=input_size, augmentation=True)
mhdb7 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='val', input_size=input_size, augmentation=True)
mhdb8 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='val', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb1 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='train', input_size=input_size, augmentation=True)
add_mhdb2 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='train', input_size=input_size, augmentation=True)
add_mhdb3 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='train', input_size=input_size, augmentation=True)
add_mhdb4 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='train', input_size=input_size, augmentation=True)

add_mhdb5 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='val', input_size=input_size, augmentation=True)
add_mhdb6 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='val', input_size=input_size, augmentation=True)
add_mhdb7 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='val', input_size=input_size, augmentation=True)
add_mhdb8 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='val', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb1 = torch.utils.data.ConcatDataset([mhdb1, add_mhdb1])
concat_mhdb2 = torch.utils.data.ConcatDataset([mhdb2, add_mhdb2])
concat_mhdb3 = torch.utils.data.ConcatDataset([mhdb3, add_mhdb3])
concat_mhdb4 = torch.utils.data.ConcatDataset([mhdb4, add_mhdb4])

concat_mhdb5 = torch.utils.data.ConcatDataset([mhdb5, add_mhdb5])
concat_mhdb6 = torch.utils.data.ConcatDataset([mhdb6, add_mhdb6])
concat_mhdb7 = torch.utils.data.ConcatDataset([mhdb7, add_mhdb7])
concat_mhdb8 = torch.utils.data.ConcatDataset([mhdb8, add_mhdb8])


print(concat_mhdb1.__getitem__(0)[0].size(), concat_mhdb1.__len__())
print(concat_mhdb5.__getitem__(0)[0].size(), concat_mhdb5.__len__())

torch.Size([3, 300, 300]) 2545
torch.Size([3, 300, 300]) 424


In [5]:
mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb1, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb2, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb3, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb4, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

val_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb5, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb6, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb7, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb8, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

## 2. Define a GoogLeNet

In [6]:
from googlenet import *

net = GoogLeNet(in_channel=3, num_classes=num_classes, aux_block=True).cuda()

for i in net.named_children():
    print(i)
#param_list = list(net.children())
#print(param_list)

('conv_block1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool1', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('conv_block2', Sequential(
  (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('conv_block3', Sequential(
  (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool2', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('inception3a', InceptionBlock(
  (conv_1x1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace)


## 3. Define a Loss function and optimizer

In [7]:
from torch.optim.lr_scheduler import StepLR

#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss() #error
#criterion = torch.nn.BCELoss() #error
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=SGD_momentum)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=8, gamma=0.96)

## 4. Train the network

In [8]:
times = 1
aux_loss_weight = 0.3

for epoch in range(epochs):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(mhdb_batch1):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda().long()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        aux_1, aux_2, outputs = net(inputs) #tuple: (aux1, aux2, input)
        
        aux_1 = aux_1.reshape(batch_size, num_classes)
        aux_2 = aux_2.reshape(batch_size, num_classes)
        outputs = outputs.reshape(batch_size, num_classes)
        
        #print(aux_1.size())
        #print(aux_2.size())
        #print(outputs.size())
        
        #print("labels", labels.size())
        
        loss1 = criterion(aux_1, labels)
        loss2 = criterion(aux_2, labels)
        loss3 = criterion(outputs, labels)
        
        loss = aux_loss_weight*loss1 + aux_loss_weight*loss2 + loss3
        loss.backward()
        scheduler.step() #학습이 안됨.
        ##optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % times == 0:
        print('[%d] loss: %.3f' %(epoch, running_loss / times))
        running_loss = 0.0
        #torch.save(model,'./data/cifar_model.pkl')

print('\nFinished Training')

[0] loss: 27.776
[1] loss: 27.772
[2] loss: 27.764
[3] loss: 27.771
[4] loss: 27.759
[5] loss: 27.772
[6] loss: 27.768
[7] loss: 27.771
[8] loss: 27.762
[9] loss: 27.771
[10] loss: 27.766
[11] loss: 27.762
[12] loss: 27.766
[13] loss: 27.770
[14] loss: 27.766
[15] loss: 27.776
[16] loss: 27.767
[17] loss: 27.771
[18] loss: 27.768
[19] loss: 27.772

Finished Training


## 5. Test the network on the test data

In [9]:
correct = 0
total = 0
cpu_net = net.cpu()

with torch.no_grad():
    cpu_net.eval()
    for data in val_mhdb_batch1:
        images, labels = data
        images = images.cpu() #.cuda()
        labels = labels.cpu().long()
        
        outputs = cpu_net(images)#.reshape(batch_size,1)
        #print(labels.size())
        outputs = outputs.reshape(batch_size, num_classes)
        _, predicted = torch.max(outputs, dim=1)
        #print(predicted.size())
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' %(concat_mhdb5.__len__(), 100 * correct / total)) #more than 50 percent

Accuracy of the network on the 424 test images: 32 %


In [11]:
correct1 = 0
correct2 = 0
correct3 = 0
total = 0

with torch.no_grad():
    net.train()
    for data in val_mhdb_batch1:
        images, labels = data
        images = images.cpu() #.cuda()
        labels = labels.cpu().long() #.cuda()
        
        aux_1, aux_2, outputs = net(images) #tuple: (aux1, aux2, input)
        aux_1 = aux_1.reshape(batch_size, num_classes)
        aux_2 = aux_2.reshape(batch_size, num_classes)
        outputs = outputs.reshape(batch_size, num_classes)
        
        _, predicted1 = torch.max(aux_1, dim=1)
        _, predicted2 = torch.max(aux_2, dim=1)
        _, predicted3 = torch.max(outputs, dim=1)
        #print(predicted.size())
        
        total += labels.size(0)
        correct1 += (predicted1 == labels).sum().item()
        correct2 += (predicted2 == labels).sum().item()
        correct3 += (predicted3 == labels).sum().item()

print('Accuracy_1 of the network on the %d test images: %d %%' %(concat_mhdb5.__len__(), 100 * correct1 / total))
print('Accuracy_2 of the network on the %d test images: %d %%' %(concat_mhdb5.__len__(), 100 * correct2 / total))
print('Accuracy_3 of the network on the %d test images: %d %%' %(concat_mhdb5.__len__(), 100 * correct3 / total))

Accuracy_1 of the network on the 424 test images: 61 %
Accuracy_2 of the network on the 424 test images: 66 %
Accuracy_3 of the network on the 424 test images: 33 %
